In [11]:
import sys
import os

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)


In [12]:
import tensorflow as tf
import numpy as np
from src.model.model_building import RankingModel, UserTower, ItemTower

/Users/marysiapacocha/Desktop/books/goodreads-recs-extension/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_path = 'model_weights.h5'

user_tower = UserTower()
item_tower = ItemTower()
model = RankingModel(user_tower, item_tower)
embedding_dim = 1536


dummy_input_combined = {
    "user_history": tf.zeros((1, 50, 1536), dtype=tf.float32),
    "history_ratings": tf.zeros((1, 50), dtype=tf.float32),
    "book_emb": tf.zeros((1, 1536), dtype=tf.float32),
    "avg_rating": tf.zeros((1,), dtype=tf.float32)
}

_ = model(dummy_input_combined)

model.load_weights('../model_weights.h5')
model.summary()

Wagi załadowane pomyślnie!
Model: "ranking_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 user_tower_3 (UserTower)    multiple                  3493248   
                                                                 
 item_tower_3 (ItemTower)    multiple                  412032    
                                                                 
 sequential_11 (Sequential)  (1, 1)                    75777     
                                                                 
 ranking_3 (Ranking)         multiple                  0 (unused)
                                                                 
Total params: 3981057 (15.19 MB)
Trainable params: 3980289 (15.18 MB)
Non-trainable params: 768 (3.00 KB)
_________________________________________________________________


In [50]:
import pandas as pd

user_history_df = pd.read_csv('../src/scraper/user_history.csv')
user_history_df.head(50)

,book_id,title,author,rating,genres,description
0,246813919-tylko-mi-okna-zostawcie-otwarte,Tylko mi okna zostawcie otwarte,"Czernek-Pasierbek, Paulina",4,"Fiction, Fantasy, Thriller, Poetry, Classics, ...",Marianna Roszkowska jest niesamowicie silna. N...
1,241722107-jak-nie-zabi-em-swojego-ojca-i-jak-b...,Jak nie zabiłem swojego ojca i jak bardzo tego...,"Pakuła, Mateusz",3,"Fiction, Poland, Favorites, Biography, Polish ...","Szczera, intymna do granic, groteskowa, brutal..."
2,43509266-internat,Internat,"Zhadan, Serhiy",4,"Fiction, Ukraine, War, Ukrainian Literature, F...",U Żadana koniec znanego świata okazuje się wst...
3,231336659-muzyka-dla-duch-w,Muzyka dla duchów,An Yu,5,"Fiction, Literary Fiction, Fantasy, China, Aud...",Pewnej nocy niespełna trzydziestoletnią Song Y...
4,246473316-spotkajmy-si-kiedy-b-dzie-adna-pogoda,"Spotkajmy się, kiedy będzie ładna pogoda","Lee, Do Woo",3,"Fiction, Romance, Asia, Slice Of Life, France,...",Bestsellerowy romans literacki z Korei.Subteln...
5,208155402-gifted,Gifted,"Suzuki, Suzumi",4,"Fiction, Japanese Literature, Asia, Japan, Fan...",A moving portrayal of a troubled mother–daught...
6,244885025-blisko-domu,Blisko domu,"Magee, Michael",4,"Fiction, Book Club, Irish Literature, Ireland,...",Szczery i poruszający obraz pokolenia stojąceg...
7,224003127-the-ten-year-affair,The Ten Year Affair,"Somers, Erin",5,"Fiction, Romance, Book Club, Adult, Fantasy, A...",“The best book about adultery sinceMadame Bova...
8,246777849-kuchnia,Kuchnia,"Yoshimoto, Banana",4,"Fiction, Short Stories, Japanese Literature, A...","Książka, z którą utożsamiają się pokolenia. Ta..."
9,242259021-jutro-ju-by-o,Jutro już było,"Bijan, Marta",3,"Fiction, Young Adult, Polish Literature, Summe...",Jak koniec świata to tylko z Martą Bijan! Sięg...


In [ ]:
from src.data.get_embeddings import get_embeddings_batch

embeddings = []

for index, row in user_history_df.iterrows():
    title = row['title']
    genres = row['genres']
    description = row['description']
    text = f"{title} {description} {genres}"
    embedding = get_embeddings_batch(text)
    embeddings.append(embedding)
embeddings = np.array(embeddings)

In [ ]:
ratings = user_history_df['rating'].values.astype(np.float32)

user_history_tensor = tf.expand_dims(tf.convert_to_tensor(embeddings, dtype=tf.float32), 0)
history_ratings_tensor = tf.expand_dims(tf.convert_to_tensor(ratings, dtype=tf.float32), 0)

print(f"Kształt historii: {user_history_tensor.shape}")   
print(f"Kształt ocen: {history_ratings_tensor.shape}")    

Kształt historii: (1, 50, 1, 1536)
Kształt ocen: (1, 50)


In [33]:
target_title = "Katabasis"
target_author = "R.F. Kuang"
target_desc = "Dante’s Inferno meets Susanna Clarke’s Piranesi in this all-new dark academia fantasy from R. F. Kuang, the #1 New York Times bestselling author of Babel and Yellowface, in which two graduate students must put aside their rivalry and journey to Hell to save their professor’s soul—perhaps at the cost of their own. Katabasis, noun, Ancient Greek: The story of a hero’s descent to the underworld Alice Law has only ever had one goal: to become one of the brightest minds in the field of Magick. She has sacrificed everything to make that a reality: her pride, her health, her love life, and most definitely her sanity. All to work with Professor Jacob Grimes at Cambridge, the greatest magician in the world. That is, until he dies in a magical accident that could possibly be her fault. Grimes is now in Hell, and she’s going in after him. Because his recommendation could hold her very future in his now incorporeal hands and even death is not going to stop the pursuit of her dreams…. Nor will the fact that her rival, Peter Murdoch, has come to the very same conclusion. With nothing but the tales of Orpheus and Dante to guide them, enough chalk to draw the Pentagrams necessary for their spells, and the burning desire to make all the academic trauma mean anything, they set off across Hell to save a man they don’t even like. But Hell is not like the storybooks say, Magick isn’t always the answer, and there’s something in Alice and Peter’s past that could forge them into the perfect allies…or lead to their doom."
target_genres = "Fantasy, Dark Academia, Fiction, Romance"
target_avg_rating = 3.77

target_text = f"{target_title} {target_desc} {target_genres}"
target_emb = get_embeddings_batch(target_text) 

book_emb_tensor = tf.expand_dims(tf.convert_to_tensor(target_emb, dtype=tf.float32), 0)
avg_rating_tensor = tf.convert_to_tensor([target_avg_rating], dtype=tf.float32)

2026-02-16 18:38:16,779 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [63]:
book_emb_tensor.shape

TensorShape([1, 1, 1536])

In [ ]:
user_history_fixed = tf.squeeze(user_history_tensor) 
if len(user_history_fixed.shape) == 2: 
    user_history_fixed = tf.expand_dims(user_history_fixed, 0) 

book_emb_fixed = tf.squeeze(book_emb_tensor)
if len(book_emb_fixed.shape) == 1: 
    book_emb_fixed = tf.expand_dims(book_emb_fixed, 0) 
    
model_input = {
    "user_history": user_history_fixed,
    "history_ratings": history_ratings_tensor, 
    "book_emb": book_emb_fixed,
    "avg_rating": tf.reshape(avg_rating_tensor, [1]) 
}

prediction = model(model_input)

print(f"\n--- WYNIK DLA KSIĄŻKI: {target_title} ---")
print(f"Przewidywany score: {prediction.numpy()[0][0]:.4f}")


--- WYNIK DLA KSIĄŻKI: Katabasis ---
Przewidywany score: 2.4488


In [ ]:
target_title_2 = "Ogrodnik i śmierć"
target_author_2 = "Georgi Gospodinov"
target_genres_2 = "Fiction, Bulgarian Literature, Literary Fiction, Philosophy, Death, Contemporary"
target_avg_rating_2 = 4.49

target_desc_2 = "Relacja między ojcem i synem ukazana poprzez egzystencjalne studium choroby, odchodzenia i żałoby. Narrator imieniem Georgi obserwuje ostatni miesiąc życia ojca. Dokumentuje wszystkie drobne i istotne zdarzenia. Zwłaszcza ojcowską pasję do roślin – dzięki którym, po zwalczeniu raka nabrał nowych sił. Ogród trzyma go przy życiu, przy ziemi. Gospodinow luźnymi zapiskami z utraty buduje historię uniwersalnego doświadczenia relacji między ojcem i synem. Literacka obserwacja pełna czułości, która mówi o tym, co dzieje się przed śmiercią."

text_for_emb_2 = f"{target_title_2} {target_desc_2} {target_genres_2}"

target_emb_2 = get_embeddings_batch(text_for_emb_2)

2026-02-16 18:52:17,283 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [56]:
book_emb_fixed_2 = tf.convert_to_tensor(target_emb_2, dtype=tf.float32)
if len(book_emb_fixed_2.shape) > 2:
    book_emb_fixed_2 = tf.squeeze(book_emb_fixed_2)
if len(book_emb_fixed_2.shape) == 1:
    book_emb_fixed_2 = tf.expand_dims(book_emb_fixed_2, 0)

model_input_2 = {
    "user_history": user_history_fixed,              
    "history_ratings": history_ratings_tensor,         
    "book_emb": book_emb_fixed_2,                     
    "avg_rating": tf.constant([target_avg_rating_2], dtype=tf.float32)
}

prediction_2 = model(model_input_2)
score_2 = prediction_2.numpy()[0][0]

print(f"--- WYNIK DLA KSIĄŻKI: {target_title_2} ---")
print(f"Przewidywany score: {score_2:.4f}")

--- WYNIK DLA KSIĄŻKI: Ogrodnik i śmierć ---
Przewidywany score: 2.7774


In [ ]:
target_title_3 = "Chłopki. Opowieść o naszych babkach"
target_author_3 = "Joanna Kuciel-Frydryszak"
target_genres_3 = "Nonfiction, History, Polish Literature, Reportage, Feminism, Poland"
target_avg_rating_3 = 4.42

target_desc_3 = "Opowieść zza drugiej strony drzwi chłopskiej chałupy. Wiejskie kobiety: harujące od świtu do nocy gospodynie, folwarczne wyrobnice, mamki. Marzące o własnym łóżku, butach, szkole. Joanna Kuciel-Frydryszak daje wiejskim kobietom głos, by opowiedziały o swoim życiu: codziennym znoju, lękach i marzeniach. Mocna lektura pokazująca siłę kobiet i ich walkę w patriarchalnym społeczeństwie."

text_for_emb_3 = f"{target_title_3} {target_desc_3} {target_genres_3}"
target_emb_3 = get_embeddings_batch(text_for_emb_3)

2026-02-16 18:54:38,041 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
book_emb_fixed_3 = tf.convert_to_tensor(target_emb_3, dtype=tf.float32)
if len(book_emb_fixed_3.shape) == 1:
    book_emb_fixed_3 = tf.expand_dims(book_emb_fixed_3, 0)

model_input_3 = {
    "user_history": user_history_fixed,
    "history_ratings": history_ratings_tensor,
    "book_emb": book_emb_fixed_3,
    "avg_rating": tf.constant([target_avg_rating_3], dtype=tf.float32)
}

prediction_3 = model(model_input_3)
print(f"--- WYNIK DLA KSIĄŻKI: {target_title_3} ---")
print(f"Przewidywany score: {prediction_3.numpy()[0][0]:.4f}")

--- WYNIK DLA KSIĄŻKI: Chłopki. Opowieść o naszych babkach ---
Przewidywany score: 2.7102


In [65]:
embedding_matrix = np.load('../src/data/embeddings_matrix.npy')

def get_top_recommendations(model, user_history_tensor, history_ratings_tensor, embedding_matrix, n=10):
    scores = []
    
    batch_size = 512
    num_books = embedding_matrix.shape[0]
    
    print(f"Obliczam wyniki dla {num_books} książek...")
    
    for i in range(0, num_books, batch_size):
        end_idx = min(i + batch_size, num_books)
        batch_embs = embedding_matrix[i:end_idx]
        
     
        current_batch_len = end_idx - i
        user_hist_batch = tf.tile(user_history_tensor, [current_batch_len, 1, 1])
        user_rat_batch = tf.tile(history_ratings_tensor, [current_batch_len, 1])
        

        book_embs_batch = tf.convert_to_tensor(batch_embs, dtype=tf.float32)
        avg_ratings_batch = tf.fill([current_batch_len], 4.0) 
        model_input = {
            "user_history": user_hist_batch,
            "history_ratings": user_rat_batch,
            "book_emb": book_embs_batch,
            "avg_rating": avg_ratings_batch
        }
        
        batch_scores = model(model_input)
        scores.extend(batch_scores.numpy().flatten())
    
    results = pd.DataFrame({
        'book_id': range(num_books),
        'score': scores
    })
    
    top_results = results.sort_values(by='score', ascending=False).head(n + 5)
    
    return top_results

top_10 = get_top_recommendations(model, user_history_fixed, history_ratings_tensor, embedding_matrix)


print("\n--- TWOJE TOP 10 REKOMENDACJI ---")
print(top_10)

Obliczam wyniki dla 10000 książek...

--- TWOJE TOP 10 REKOMENDACJI ---
      book_id     score
266       266  3.397719
118       118  3.386821
458       458  3.369529
142       142  3.363594
24         24  3.341062
140       140  3.333567
4016     4016  3.317389
6281     6281  3.316935
1470     1470  3.315808
20         20  3.310098
1789     1789  3.306745
1461     1461  3.295141
2540     2540  3.292366
8503     8503  3.291038
214       214  3.290251


In [ ]:
books = pd.read_csv(
        "https://raw.githubusercontent.com/malcolmosh/goodbooks-10k/master/books_enriched.csv",
    )
top_ids = [266, 118, 458, 142, 24]

recommendations = books[books['book_id'].isin(top_ids)][['book_id', 'title', 'average_rating']]
print(recommendations.sort_values(by='book_id'))

     book_id                                              title  \
20        24  Harry Potter and the Goblet of Fire (Harry Pot...   
106      118                                  The Joy Luck Club   
128      142  The Pillars of the Earth (The Kingsbridge Seri...   
240      266                    Kiss the Girls (Alex Cross, #2)   
421      458                                     Station Eleven   

     average_rating  
20             4.53  
106            3.90  
128            4.29  
240            3.93  
421            4.02  
